In [50]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, Lasso, Ridge



from sklearn.linear_model import LassoCV, Lasso
import numpy as np
from sklearn.datasets import make_regression



from csv_reader import *
from csv_setup import *

In [93]:
bike_sharing_df = get_bike_sharing_df_clean()
bike_sharing_df.columns


print(bike_sharing_df.columns)


Index(['hr', 'holiday', 'workingday', 'weathersit', 'atemp', 'hum',
       'windspeed', 'cnt', 'no_snow', 'medium_snow', 'heavy_snow'],
      dtype='object')


In [ ]:



Y = bike_sharing_df.cnt
X = bike_sharing_df.loc[:].drop(columns=["cnt"])




X_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33, random_state = 1)
print(X_train)

n_folds = 10
n_alphas = 200
alphas = np.logspace(-5, 5, n_alphas) #grid search for lambda (aka 'alpha')

parameters = { 'lasso__alpha':alphas}

scaler = StandardScaler()
lasso = Lasso(normalize=True,tol=2,fit_intercept=True) #we will pass through alpha values later
pipe = make_pipeline(scaler, lasso)

gs = GridSearchCV(pipe, parameters, scoring='neg_mean_squared_error', cv=n_folds)
gs.fit(X_train,y_train)

alphas = np.logspace(-5, 5, n_alphas) #grid search for lambda (aka 'alpha')

coefs = []
# print(pipe.get_params().keys()) #reports input names can set (look for the hyper-parameters)
parameters = {'lasso__alpha':alphas}
# model = GridSearchCV(pipe, parameters, scoring='neg_mean_squared_error', cv=n_folds)
# model.fit(X_train,y_train)
for a in alphas:
    pipe.set_params(lasso__alpha=a)
    pipe.fit(X_train, y_train)
    coefs.append(pipe.named_steps['lasso'].coef_)

plt.figure()
ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')

plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('Coefficients');


# plt.figure()
# ax = plt.gca()
# ax.plot(alphas, coefs,'o')
# ax.set_xscale('log')
#
# plt.axis('tight')
# plt.xlabel('alpha')
# plt.ylabel('Coefficients')
# plt.title('Zoomed in')
# plt.xlim(.01,100)


       hr  holiday  workingday  weathersit   atemp   hum  windspeed  no_snow  \
7445   20        1           0           1  0.3182  0.39     0.1940        0   
13512   5        0           0           1  0.5455  0.88     0.0000        1   
6158    4        0           1           2  0.4848  0.77     0.1642        1   
13205  10        0           1           2  0.6061  0.83     0.1940        1   
1616   20        0           0           1  0.4091  0.58     0.1045        0   
...    ..      ...         ...         ...     ...   ...        ...      ...   
10955  15        0           1           1  0.4848  0.27     0.2836        0   
17289   6        0           1           1  0.2121  0.64     0.2836        0   
5192    6        0           1           1  0.6667  0.64     0.0000        1   
12172   9        0           0           1  0.6061  0.83     0.2239        1   
235     2        0           1           2  0.1515  0.55     0.1940        0   

       medium_snow  heavy_snow  
7445  

In [ ]:
# coefficients {𝛽1,…,𝛽𝑝} for the last fitted model in the above for loop
pipe.named_steps['lasso'].coef_

In [ ]:
# B0
pipe.named_steps['lasso'].intercept_

In [ ]:
# Average of Y's
np.mean(y_train)

In [ ]:
# Best alpha
best_alpha = gs.best_params_['lasso__alpha']
print(best_alpha)

In [ ]:
scaler = StandardScaler()
lasso = Lasso(alpha=best_alpha,normalize=False,fit_intercept=True) #we will pass through alpha values later
pipe = make_pipeline(scaler, lasso)
pipe.fit(X_train, y_train)

best_coef_lasso = pipe.named_steps['lasso'].coef_

# print('The final set of coefficients: ')
print(best_coef_lasso)

In [ ]:
print('\nCoefficients  [lasso]')
for i in range(0,10):
    print('   Feature %2i:  %.2f'%(i,\
                    best_coef_lasso[i]))

In [ ]:
np.count_nonzero(best_coef_lasso)

In [ ]:
y_pred = pipe.predict(x_test)
mse_test_using_lasso = mean_squared_error(y_test,y_pred)
# print(mse_test_using_coef_true)
print('\nTest set MSE using Lasso coefficients: %.2f'%mse_test_using_lasso)

In [ ]:

y_train_pred = lm.predict(sm.add_constant(X_train))
y_test_pred = lm.predict(sm.add_constant(x_test))

print('Train Score: ')
print(r2_score(y_train, y_train_pred))

print('\nTest Score: ')
print(r2_score(y_test, y_test_pred))